In [1]:
import os
import json
from shutil import copy2
from collections import defaultdict

In [ ]:
# Caminhos
json_path = r"\DENTEX\training_data\quadrant-enumeration-disease\train_quadrant_enumeration_disease.json"
images_dir = r"\DENTEX\training_data\quadrant-enumeration-disease\xrays"
output_dir = "dentex_yolo"
img_out = os.path.join(output_dir, "images", "train")
label_out = os.path.join(output_dir, "labels", "train")
os.makedirs(img_out, exist_ok=True)
os.makedirs(label_out, exist_ok=True)

In [11]:
# Carregar JSON
with open(json_path) as f:
    data = json.load(f)

In [12]:
# Mapeamento de imagens
id_to_image = {img["id"]: img for img in data["images"]}
annotations_by_image = defaultdict(list)
classes_set = set()

In [13]:
for ann in data["annotations"]:
    q = data["categories_1"][ann["category_id_1"]]["name"]
    n = data["categories_2"][ann["category_id_2"]]["name"]
    d = data["categories_3"][ann["category_id_3"]]["name"]
    label = f"{q}{n}_{d}"
    classes_set.add(label)
    annotations_by_image[ann["image_id"]].append((ann["bbox"], label))

In [14]:
# Indexar classes
classes = sorted(classes_set)
class_to_id = {name: idx for idx, name in enumerate(classes)}

In [16]:
# Salvar labels YOLO
for image_id, anns in annotations_by_image.items():
    img = id_to_image[image_id]
    width, height = img["width"], img["height"]
    filename = img["file_name"]
    txt_path = os.path.join(label_out, filename.replace(".png", ".txt"))

    with open(txt_path, "w") as f:
        for bbox, label in anns:
            x, y, w, h = bbox
            xc = (x + w / 2) / width
            yc = (y + h / 2) / height
            w /= width
            h /= height
            class_id = class_to_id[label]
            f.write(f"{class_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")

    # Copiar imagem
    copy2(os.path.join(images_dir, filename), os.path.join(img_out, filename))

In [17]:
# Salvar classes.txt
with open(os.path.join(output_dir, "classes.txt"), "w") as f:
    for name in classes:
        f.write(name + "\n")

In [ ]:

# Salvar data.yaml
with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(f'train: {os.path.abspath(img_out)}\n')
    f.write(f'val: {os.path.abspath(img_out)}\n\n')  # pode criar split depois
    f.write(f'nc: {len(classes)}\n')
    f.write("names:\n")
    for name in classes:
        f.write(f'  - {name}\n')

print(f'✅ Conversão concluída. Total de classes: {len(classes)}')

✅ Conversão concluída. Total de classes: 97
